In [36]:
# pip install pymupdf4llm

In [55]:
import numpy as np
import openai
import psycopg2
import pymupdf4llm


from pgvector.psycopg2 import register_vector
from psycopg2.extras import execute_values
from loguru import logger
from jobspy import scrape_jobs

from dotenv import load_dotenv
load_dotenv()

True

## Read Resume Data

In [56]:
file_path = "/mnt/c/Users/user/OneDrive/Desktop/github-projects/reflex-recommendation-jobs/notebooks/matinsajadi.pdf"

In [57]:
md_text = pymupdf4llm.to_markdown(file_path)

Processing /mnt/c/Users/user/OneDrive/Desktop/github-projects/reflex-recommendation-jobs/notebooks/matinsajadi.pdf...
[                                        ] (0/2===================[====================                    ] (1/2===================[========================================] (2/2]


### Connect To Postgres And retrive top 3 similar jobs

In [58]:
connection = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="mypass",
    host="localhost",
    port="5432"
)


In [59]:
# Helper function: get embeddings for a text
def get_embeddings(text):
    response = openai.Embedding.create(
        model="text-embedding-ada-002",
        input = text.replace("\n"," ")
    )
    embedding = response['data'][0]['embedding']
    return embedding

In [60]:
#Helper function: Get top 3 most similar documents from the database
def get_top3_similar_docs(query, conn):
    query_embedding = get_embeddings(query)
    embedding_array = np.array(query_embedding)
    # Register pgvector extension
    register_vector(conn)
    cur = conn.cursor()
    # Get the top 3 most similar documents using the KNN <=> operator
    cur.execute("SELECT * FROM embeddings ORDER BY embedding <=> %s LIMIT 3", (embedding_array,))
    top3_docs = cur.fetchall()
    return top3_docs

In [61]:
query = f"{md_text}"

In [62]:
connection.commit()

In [63]:
get_top3_similar_docs(query=query, conn=connection)

[(39,
  'indeed',
  'https://www.indeed.com/viewjob?jk=b3cd84e3ea273500',
  'Software Developer 1',
  'Stanford University',
  'Stanford, CA, US',
  '2024-11-08',
  'yearly',
  107423.0,
  133600.0,
  'USD',
  'false',
  "The Dubra Lab at Stanford University is seeking a highly motivated software developer to create an automated ophthalmic image processing and analysis pipeline, as well as custom novel retinal imaging instrumentation. The resulting software and hardware will be deployed across premier academic centers in the US and Europe to advance our understanding, diagnosing and management of ocular, neurodegenerative and systemic diseases and conditions. This will require the implementation and optimization of novel and sophisticated image algorithms in C\\+\\+, Python, LabView, OpenGL and CUDA. The candidate must have a bachelor's degree in Computer Science or Computer engineering. Expertise in designing, developing, testing, and deploying C\\+\\+ applications using Microsoft Vis